# Saurabh Pradhan
## 277643

## Neural network setup

## The general setup of neural network is as follows :-
The input to the neural network is a vector of dimension (m , 1) where m it the total no of features, bias excluded.<br /> The dimensions of bias is (m, 1)
From m dimensional layer to other n dimensional layer a weight matrix is attached. The dimension of this weight matrix is (m, n) <br/>

In [34]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Loss Functions

In [35]:
def squaredError(y, yPrediction, derivative = False):
    if derivative == False:
        return np.sum(((y - yPrediction) ** 2))
    else:
        return -1 * (y - yPrediction)

## Activation Functions

In [36]:
import numpy as np

def relu(x, derivative = False):
    if derivative == False:
        return np.maximum(x, 0)
    else:
        """Deep copy"""
        x = x.copy()
        x[x<0] = 0
        x[x>0] = 1
        return x

def linearActivation(x, derivative = False):
    if derivative == False:
        return x
    else:
        return 1

In [37]:
def L1Regularization(gradients):
    """Deep copy"""
    gradients = gradients.copy()
    gradients[gradients > 0] = 1
    gradients[gradients <= 0] = -1
    
    return gradients

## Neural network

In [38]:
from enum import Enum

class STRUCTURE(Enum):
    WEIGHTMATRIX = "weightMatrix"
    ACTIVATIONFUNCTION = "activationFunction"
    BIAS = "bias"
    INPUTLAYER = "inputLayer"
    REGULARIZATION = "regularization"
    REGULARIZATIONPENALTY = "regularizationPenalty"
    

class NeuralNetwork:
    
    networkStructure = None
    lossFunction = None
            
    def __init__(self, networkStructure, lossFunction):
        self.networkStructure = networkStructure
        self.lossFunction = lossFunction
    
    def forwardPropogationStep(self, x, retainIntermediateValues = False):
        output = x
        intermediateValues = []
        for layer in self.networkStructure:
                aggregation = np.dot(output, layer[STRUCTURE.WEIGHTMATRIX]) + layer[STRUCTURE.BIAS]
                if retainIntermediateValues == True:
                    intermediateValues.append([output, aggregation])

                output = layer[STRUCTURE.ACTIVATIONFUNCTION](aggregation)
        return output, intermediateValues

    def backPropogationStep(self, x, y, learningRate):
        prediction, intermediateValues = self.forwardPropogationStep(x, True)
        loss = self.lossFunction(y, prediction)
        print("Calculated loss at end of forward propogation " +str(loss))
        backPropogationComponent = self.lossFunction(y, prediction, derivative = True)
        for layer, intermediateValue in zip(reversed(self.networkStructure), reversed(intermediateValues)):
            
            backPropogationComponent = backPropogationComponent * layer[STRUCTURE.ACTIVATIONFUNCTION](intermediateValue[1], 
                                                                                            derivative = True)
            """Update weights in current layer"""
            weightGradient = intermediateValue[0].T * backPropogationComponent
            biasGradient = 1 * backPropogationComponent
            
            regularization = 0
            if STRUCTURE.REGULARIZATION in layer:
                regularization = layer[STRUCTURE.REGULARIZATION](weightGradient)
                regularization = regularization * layer[STRUCTURE.REGULARIZATIONPENALTY]
            
            """Dirty update"""
            layer[STRUCTURE.WEIGHTMATRIX] = layer[STRUCTURE.WEIGHTMATRIX] - learningRate * (weightGradient +
                                                                                            regularization)
            layer[STRUCTURE.BIAS] = layer[STRUCTURE.BIAS] - learningRate * biasGradient
                       
            
            if layer[STRUCTURE.INPUTLAYER] == True:
                break
            
            """Update in the previous layer depends on the output of activation function"""
            backPropogationComponent = backPropogationComponent * layer[STRUCTURE.WEIGHTMATRIX].T

## Without regularization

In [39]:
structure = [
    {
        STRUCTURE.WEIGHTMATRIX:np.array([-1, 2, 0, -2]).reshape(2, 2),
        STRUCTURE.BIAS: np.ones(2).reshape(1, 2),
        STRUCTURE.ACTIVATIONFUNCTION : relu,
        STRUCTURE.INPUTLAYER:True
    },
    {
        STRUCTURE.WEIGHTMATRIX:np.array([-1, 2]).reshape(2, 1),
        STRUCTURE.BIAS: np.ones(1).reshape(1, 1),
        STRUCTURE.ACTIVATIONFUNCTION : linearActivation,
        STRUCTURE.INPUTLAYER:False
    }
]

inputVector = np.array([-1, 1]).reshape(1, -1) 
nn = NeuralNetwork(structure, squaredError)
output = nn.forwardPropogationStep(inputVector)[0][0]
print("The initial output is " + str(output))
print("The loss is "+ str(squaredError(2, output)))

The initial output is [-1.]
The loss is 9.0


In [40]:
nn.backPropogationStep(inputVector, 2, 0.01)
output = nn.forwardPropogationStep(inputVector)
output = nn.forwardPropogationStep(inputVector)[0][0]
print("New output is " + str(output))
print("New loss is "+ str(squaredError(2, output)))

Calculated loss at end of forward propogation 9.0
New output is [-0.770476]
New loss is 7.675537266576


In [41]:
for layer in nn.networkStructure:
    print("Bias weight")
    print(layer[STRUCTURE.BIAS])
    print("Weight")
    print(layer[STRUCTURE.WEIGHTMATRIX])
    print("\n\n\n")

Bias weight
[[0.9718 1.    ]]
Weight
[[-0.9718  2.    ]
 [-0.0282 -2.    ]]




Bias weight
[[1.03]]
Weight
[[-0.94]
 [ 2.  ]]






## With regularization

In [42]:
structure = [
    {
        STRUCTURE.WEIGHTMATRIX:np.array([-1, 2, 0, -2]).reshape(2, 2),
        STRUCTURE.BIAS: np.ones(2).reshape(1, 2),
        STRUCTURE.ACTIVATIONFUNCTION : relu,
        STRUCTURE.INPUTLAYER:True,
        STRUCTURE.REGULARIZATION:L1Regularization,
        STRUCTURE.REGULARIZATIONPENALTY:0.5
    },
    {
        STRUCTURE.WEIGHTMATRIX:np.array([-1, 2]).reshape(2, 1),
        STRUCTURE.BIAS: np.ones(1).reshape(1, 1),
        STRUCTURE.ACTIVATIONFUNCTION : linearActivation,
        STRUCTURE.INPUTLAYER:False,
        STRUCTURE.REGULARIZATION:L1Regularization,
        STRUCTURE.REGULARIZATIONPENALTY:0.5
    }
]

inputVector = np.array([-1, 1]).reshape(1, -1) 
nn = NeuralNetwork(structure, squaredError)
output = nn.forwardPropogationStep(inputVector)[0][0]
print("The initial output is " + str(output))
print("The loss is "+ str(squaredError(2, output)))

The initial output is [-1.]
The loss is 9.0


In [43]:
nn.backPropogationStep(inputVector, 2, 0.01)
output = nn.forwardPropogationStep(inputVector)
output = nn.forwardPropogationStep(inputVector)[0][0]
print("New output is " + str(output))
print("New loss is "+ str(squaredError(2, output)))

Calculated loss at end of forward propogation 9.0
New output is [-0.75196975]
New loss is 7.573337504915064


In [44]:
for layer in nn.networkStructure:
    print("Bias weight")
    print(layer[STRUCTURE.BIAS])
    print("Weight")
    print(layer[STRUCTURE.WEIGHTMATRIX])
    print("\n\n\n")

Bias weight
[[0.97195 1.     ]]
Weight
[[-0.96695  2.005  ]
 [-0.03305 -1.995  ]]




Bias weight
[[1.03]]
Weight
[[-0.935]
 [ 2.005]]






After regularization the loss has reduced a little, however the difference is very less (0.01).

## Question 7

Data augmentation allows to create additional data/ synthetic data from available training data. One way is to add white noise to the audio. The amplitude of white noise should be kept low so that the words are clearly audiable. Second method is to shuffle the sound and create new data points. Other methods include streching sounds and changing the pitch of the audio clip. Also the dynamic range of autio can also be changed to generate samples.
<ul>
    <li>Time streching</li>
    <li>Pitch Shifting</li>
    <li>Dynamic range compression</li>
    <li>Background noise</li>
</ul>
These are the methods that can be used for generating synthetic data for sound dataset.

Data augmentation helps with regularization. This is because the augmented data adds prior knowledge for the model and reduces model varience interm reducing the complexicity of the model. Hence data augmentation can be used as regularization method.